In [1]:
import torch

# mの正負に応じてイジングモデルのエネルギーを計算
def energy(m, h, J):
    m = 2 * (m > 0.0) - 1
    return -torch.sum(h * m) - (torch.sum(J * torch.outer(m, m)) - torch.sum(torch.diagonal(J * torch.outer(m, m)))) / (len(m) - 1)

# エネルギー計算関数 (ベクトル化)
def f(m, h, J, tau, alpha):
    AS = -(1 - alpha) * torch.sum(torch.log(1 - m**2))
    interaction = (torch.sum(J * torch.outer(m, m)) - torch.sum(torch.diagonal(J * torch.outer(m, m)))) / (len(m) - 1)
    ising = -alpha * (torch.sum(h * m) + interaction)
    return AS + ising

# 勾配計算関数 (ベクトル化)
def update(m, h, J, tau, alpha):
    interaction_grad = -alpha * (torch.sum(J * m, dim=1) - torch.diagonal(J) * m) / (len(m) - 1)
    dif = interaction_grad + (1 - alpha) * m / (1 - m**2) - alpha * h
    return dif

# 勾配降下法 (改良版)
def gradient_descent(m, h, J, tau, alpha, max_iter=1000000, tol=1e-6):
    learning_rate = 0.01
    for iteration in range(max_iter):
        grad = update(m, h, J, tau, alpha)
        m = m - learning_rate * grad  # 勾配降下ステップ

        # 停滞のチェック（勾配が小さくなる場合）
        if torch.norm(grad) < tol:
            alpha = min(alpha + 0.01, 0.999)  # alpha を増加

        # alpha が 0.99 に達した場合、終了
        if alpha >= 0.999:
            break

    return m, energy(m, h, J)

# シード値の設定
def initialize_random_parameters(n, seed):
    torch.manual_seed(seed)
    h = torch.normal(mean=0.0, std=0.01, size=(n,))
    J = torch.normal(mean=0.0, std=1.0, size=(n, n))
    return h, J

# 初期条件
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n = 100
seed = 42
m = torch.zeros(n, device=device)
h, J = initialize_random_parameters(n, seed)
h, J = h.to(device), J.to(device)
tau = torch.tensor(1000, device=device)
alpha = torch.tensor(0.0, device=device)

# GPU が使用されているか確認
print(device)

# 勾配降下法を実行
m_final, value_final = gradient_descent(m, h, J, tau, alpha)

# 最終結果を出力
print("Final m:", m_final.cpu().numpy())
print("Final energy:", value_final.cpu().item())


cuda
Final m: [ 0.05002664  0.05315663  0.02385301 -0.0501005   0.03246869 -0.01869557
 -0.008      -0.03956174 -0.018731    0.04179331  0.00136474 -0.04216671
 -0.0308504  -0.0103564  -0.02677374  0.02824917  0.05276158 -0.00430204
 -0.02508965  0.0056492  -0.02097845  0.03438026  0.02392421  0.04209607
  0.03913037  0.04469684 -0.001118    0.04721301  0.00186956 -0.00842103
 -0.00867447  0.01708049 -0.02989651 -0.02563168 -0.00571196  0.05802053
  0.00271412 -0.01569063  0.01985935 -0.0299391  -0.04184046  0.03340454
 -0.0102529  -0.02158016 -0.03436199  0.06380051 -0.03333382 -0.00897132
 -0.02911194 -0.01545574  0.00273725  0.00513056 -0.00859242  0.03868818
 -0.01153025 -0.02404947 -0.04011608  0.00657356  0.00111681  0.0205841
 -0.00104646  0.05306929 -0.0403423   0.04022284  0.04161224  0.01306188
  0.06114554  0.0206666   0.01566026 -0.00531882 -0.02611646  0.03409257
 -0.01090705  0.01245025  0.00257256  0.00029992  0.02100495 -0.01658174
 -0.06582303 -0.02594634  0.0034457  -